In [1]:
import os
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import cv2
import glob
import re

def enlarge_and_annotate(input_folder, output_folder, coordinates, size, magnification):    
    """
    Enlarges a section of each image, places it in the bottom left corner, and draws red boxes and lines.

    Parameters:
    - input_folder: Path to the folder containing the images.
    - output_folder: Path to the folder where modified images will be saved.
    - coordinates: Tuple (x, y) indicating the top-left corner of the section to enlarge.
    - size: Tuple (width, height) indicating the size of the section to enlarge.
    - magnification: Magnification factor for enlarging the section.
    """
    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            full_path = os.path.join(input_folder, filename)

            # Open the image
            with Image.open(full_path) as img:
                draw = ImageDraw.Draw(img)
                # Define the area to crop and enlarge
                x, y = coordinates
                width, height = size
                area = (x, y, x + width, y + height)

                # Draw a red rectangle around the section to enlarge
                draw.rectangle(area, outline="white", width=2)

                # Crop the desired section and enlarge it
                cropped_section = img.crop(area).resize((int(width * magnification), int(height * magnification)), Image.ANTIALIAS)
                
                # Calculate the bottom left corner position for the enlarged section
                bottom_left_x = 0
                bottom_left_y = 0

                # Draw a red rectangle around the enlarged section
                enlarged_area = (bottom_left_x, bottom_left_y, bottom_left_x + cropped_section.width, bottom_left_y + cropped_section.height)
                draw.rectangle(enlarged_area, outline="white", width=2)

                # Draw lines indicating the enlargement
                # From the original section to the enlarged section
                draw.line([200, 0, x + height, y], fill="white", width=2)
                draw.line([0, 200, x, y + height], fill="white", width=2)

                # Paste the enlarged section in the bottom left corner
                img.paste(cropped_section, (bottom_left_x, bottom_left_y))

                # Save the modified image to the output folder
                output_path = os.path.join(output_folder, filename)
                img.save(output_path)

                #print(f"Processed {filename} and saved to {output_path}")


def calculate_average(paths):
    sum_image = None
    for path in paths:
        image = np.array(Image.open(path)).astype(np.float32)
        if sum_image is None:
            sum_image = np.zeros_like(image)
        sum_image += image
    average = sum_image / len(paths)
    return average

def apply_field_correction(images_folder, dark_ave, light_ave, output_folder, reg):

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

        # Iterate over all images in the folder and apply correction
    for image_file in os.listdir(images_folder):
        image_path = os.path.join(images_folder, image_file)
        if os.path.isfile(image_path):

            image = np.array(Image.open(image_path)).astype(np.float32)

            # If image has 4 channels and dark_ave is 2D, we need to expand dark_ave's dimensions
            if image.shape[-1] in [3, 4]:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            

            # Apply dark frame subtraction
            dark_sub_image = image - dark_ave
            
            # Normalize the light and dark-subtracted images
            light_ave_normalized = (light_ave - np.min(light_ave)) / (np.max(light_ave) - np.min(light_ave))
            #dark_sub_image_normalized = (dark_sub_image - np.min(dark_sub_image)) / (np.max(dark_sub_image) - np.min(dark_sub_image))
            
            # Apply flat field correction
            corrected_image = dark_sub_image / (reg + light_ave_normalized)  # 0.3 for USAF, 0.1 for plant 0.2 for timelapse
            #print(np.max(corrected_image), np.min(corrected_image))
            
            # normalize
            corrected_image_normalized = (corrected_image - np.min(corrected_image)) / (np.max(corrected_image) - np.min(corrected_image)) * 255

            


            # Save the corrected image
            basename = os.path.basename(image_path)
            corrected_filename = os.path.join(output_folder, os.path.splitext(basename)[0] + '_corrected.png')
            io.imsave(corrected_filename, corrected_image_normalized.astype(np.uint8))


            #print(np.max(corrected_image_normalized), np.min(corrected_image_normalized))
            #print(f"Saved corrected image to {corrected_filename}")

def normalize(images_folder, output_folder, STD=True, USAF=False):
    """
    This function processes and normalizes images from a specified folder and saves the normalized images to an output folder.

    :param images_folder: The path to the folder containing the images to be processed.
    :param output_folder: The path to the folder where the normalized images will be saved.
    :param STD: A boolean flag to determine the type of normalization. If True, standard deviation normalization is used.
                If False, min-max normalization is used. Defaults to True.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over all images in the folder
    for image_file in os.listdir(images_folder):
        image_path = os.path.join(images_folder, image_file)
        if os.path.isfile(image_path):
            # Open the image
            image = np.array(Image.open(image_path)).astype(np.float32)

            # Normalize the image
            mean = np.mean(image)
            std = np.std(image)

            if STD:
                if USAF:
                    normalized_img = ((image - mean) / std) * (128/2) # USAF
                else:
                    normalized_img = ((image - mean) / std) * (128 / 2) + 128
                
                normalized_img_clipped = np.clip(normalized_img, 0, 255).astype(np.uint8)
            else:
                normalized_img_clipped = ((image - np.min(image)) / (np.max(image) - np.min(image)) * 255).astype(np.uint8)

            # Save the normalized image
            basename = os.path.basename(image_path)
            corrected_filename = os.path.join(output_folder, os.path.splitext(basename)[0] + '_normalized.png')
            io.imsave(corrected_filename, normalized_img_clipped)

            #print(f"Saved normalized image to {corrected_filename}")

def numerical_sort(value):
    parts = re.compile(r'(\d+)').split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

def create_video_and_plot_intensities(image_folder, base_output_path, fps=10, interval_minutes=10):
    image_files = glob.glob(os.path.join(image_folder, '*'))
    image_files.sort(key=lambda x: int(''.join(filter(str.isdigit, x))))

    if not image_files:
        print("No images found in the specified folder.")
        return

    frame = cv2.imread(image_files[0])
    if frame is None:
        print("Failed to read the first image file.")
        return
    height, width, layers = frame.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_video_file = os.path.join(base_output_path, 'output_video.mp4')
    os.makedirs(base_output_path, exist_ok=True)
    video = cv2.VideoWriter(output_video_file, fourcc, fps, (width, height))

    mean_intensities = []
    for idx, image_file in enumerate(image_files):
        frame = cv2.imread(image_file)
        if frame is None:
            print(f"Warning: Could not read image file {image_file}")
            continue
        mean_intensity = cv2.mean(frame)[:3]  # Ignore alpha channel if present
        mean_intensities.append(np.mean(mean_intensity))

        # Add timestamp to the frame
        timestamp = f"{(idx * interval_minutes) / 60:.2f} hrs"  # Convert frame index to hours
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, timestamp, (width - 200, 30), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        video.write(frame)

    video.release()
    print(f"Video file created: {output_video_file}")

    # Plotting the intensity graph
    time_scale = np.arange(0, len(mean_intensities) * interval_minutes, interval_minutes) / 60  # Convert to hours
    plt.figure()
    plt.xlabel('Time (hours)')
    plt.ylabel('Mean Intensity')
    plt.plot(time_scale, mean_intensities)
    plt.savefig(os.path.join(base_output_path, 'mean_intensity_plot.png'))
    plt.close()
    print("Intensity plot saved.")

In [6]:
input_folder =  '/home/alingold/rdmpy_20231220/output/USAF_grid_no_focus_plant_beads'

# field correction params
dark_paths = [f'/media/alingold/MenonLab/20240119_miniscope/Dark/{file}' for file in os.listdir('/media/alingold/MenonLab/20240119_miniscope/Dark')]
#light_paths = [f'/media/alingold/MenonLab/20240119_miniscope/Bright/{file}' for file in os.listdir('/media/alingold/MenonLab/20240119_miniscope/Bright')] # this one for plant
light_paths = [f'/media/alingold/MenonLab/20240112_Pink_field/{file}' for file in os.listdir('/media/alingold/MenonLab/20240112_Pink_field')] # this one for USAF

# Calculate averages for dark and light images
dark_ave = calculate_average(dark_paths)  # Assuming dark_paths is defined
light_ave = calculate_average(light_paths)  # Assuming light_paths is defined

# enlarged cuttout params
#coordinates = (250, 100)  #  plant
#coordinates = (220, 280)  #  timelapse cross section
#coordinates = (80, 380)  #  timelapse longitudinal section
coordinates = (280, 260)  # USAF
#coordinates = (250, 300)  # greenhouse cross section
#coordinates = (450, 250)  # greenhouse longitudinal section
#coordinates = (250, 250)  # greenhouse fungus section

size = (100, 100)  # Size of the section to enlarge
magnification = 2  # Magnification factor

# video params:
fps = 10
interval_minutes = 10 #60 #1.892  # Assuming each image represents a 5-minute interval

USAF = True
measurement = False

# Normalize and enlarge ring deconvolved imgs
create_video_and_plot_intensities(input_folder, input_folder + '/video', fps, interval_minutes)
enlarge_and_annotate(input_folder, input_folder + '/enlarged', coordinates, size, magnification)
normalize(input_folder, input_folder + '/normalized', STD=False, USAF=USAF)
create_video_and_plot_intensities(input_folder + '/normalized', input_folder + '/normalized/video', fps, interval_minutes)
enlarge_and_annotate(input_folder + '/normalized', input_folder + '/normalized/enlarged', coordinates, size, magnification)


# Normalize and enlarge measurements
if measurement:
    create_video_and_plot_intensities(input_folder + '/measurements', input_folder + '/measurements/video', fps, interval_minutes)
    normalize(input_folder + '/measurements', input_folder + '/measurements/normalized', STD=False, USAF=USAF)
    create_video_and_plot_intensities(input_folder + '/measurements/normalized', input_folder + '/measurements/normalized/video', fps, interval_minutes)
    enlarge_and_annotate(input_folder + '/measurements/normalized', input_folder + '/measurements/normalized/enlarged', coordinates, size, magnification)
    enlarge_and_annotate(input_folder + '/measurements', input_folder + '/measurements/enlarged', coordinates, size, magnification)


#field correction
apply_field_correction(input_folder + '/normalized', dark_ave, light_ave, input_folder + '/normalized/corrected_0.2', reg=0.2)
normalize(input_folder + '/normalized/corrected_0.2', input_folder + '/normalized/corrected_0.2/normalized_std', STD=True, USAF=USAF)
create_video_and_plot_intensities(input_folder + '/normalized/corrected_0.2/normalized_std', input_folder + '/normalized/corrected_0.2/normalized_std/video', fps, interval_minutes)
enlarge_and_annotate(input_folder + '/normalized/corrected_0.2/normalized_std', input_folder + '/normalized/corrected_0.2/normalized_std/enlarged', coordinates, size, magnification)



Failed to read the first image file.


<ipython-input-1-c4b95b8bf32d>:42: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  cropped_section = img.crop(area).resize((int(width * magnification), int(height * magnification)), Image.ANTIALIAS)


Failed to read the first image file.
Failed to read the first image file.
